# Forecasting S&P 500 Price Movements with Machine Learning

This notebook aims to forecast daily price movements of the **S&P 500 index** using machine learning classifier **Random Forest**, combined with **resampling techniques** to address class imbalance (e.g., **SMOTE**, undersampling).

---

### Forecasting Objective

We frame this as a **binary classification problem**, where the goal is to predict whether the S&P 500 will close **above or below its intraday average** based on early trading data.

For each trading day:

1. Extract all 1-minute price candles from market open until **30 minutes before close**.
2. Compute the **average price** over this period.
3. Compare it with the **closing price of the last 1-minute candle** of the day:
   - If the final price is **above** the average → target = `1`
   - If it is **below** the average → target = `0`

This formulation allows the model to generate **intra-day trading signals** using only early and mid-day price data.

---

### Background and Translation Notes

This notebook is a **Python translation** of the original R-based methodology from [this GitHub repository](https://github.com/LuZhang907/RandomForest).  
The original code leverages **ROSE (Random OverSampling Examples)** to address class imbalance by generating synthetic samples with added noise. However, as ROSE is not natively available in Python's ecosystem, we focus on **SMOTE (Synthetic Minority Over-sampling Technique)**, which creates new samples through interpolation between existing minority instances.

---

## Environment Setup

To reproduce this project, please ensure the following dependencies are installed:

- Python
- pandas  
- numpy  
- scikit-learn 
- imbalanced-learn
- pytz
- PyWavelets 

You can install them with:

```bash
pip install -r requirements.txt
```

Or by running:

```bash
pip install pandas numpy scikit-learn imbalanced-learn pytz PyWavelets
```

---

In [28]:
# %pip install -r requirements.txt

## 1. Library Imports
This section loads all required libraries for data manipulation, model training, evaluation, and resampling. Ensure that all dependencies listed in the environment section are installed.

In [29]:
import os, glob
import numpy as np
import pandas as pd
from datetime import time
from pytz import timezone
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE

## 2. Data Loading and Aggregation
This section loads the raw CSV files (presumably minute-level S&P 500 stock data), aggregates them, and converts them into a usable dataframe.

In [30]:
# 1) —— Data Import & Preprocessing —— #

DATA_DIR = "../SPX"
OUTPUT_FEATURE_CSV = "../csvfiles_python/features_360_raw.csv"
OUTPUT_ALLSET_CSV  = "../csvfiles_python/allSet_360_raw.csv"

def load_data(path):
    files = glob.glob(os.path.join(path, "*.txt"))
    dfs = []
    for f in sorted(files):
        df = pd.read_csv(f, names=["DateTime","Open","High","Low","Close"],
                         sep=",", parse_dates=["DateTime"])
        dfs.append(df)
    spx = pd.concat(dfs, ignore_index=True)
    # localize to EST
    spx["DateTime"] = spx["DateTime"].dt.tz_localize(timezone("US/Eastern"))
    spx.set_index("DateTime", inplace=True)
    # filter trading hours 9:30–16:00
    return spx.between_time("09:30","16:00")

spx = load_data(DATA_DIR)

# Drop duplicates
spx = spx[~spx.index.duplicated(keep='first')]

## 3. Target Variable Construction
A binary classification target is defined based on future price movement (e.g., 1 if the price increases, 0 if not).

In [31]:
# 2) —— Create daily labels —— #

# group by calendar date
spx["Date"] = spx.index.date
groups = spx.groupby("Date")

# last-minute close price per day
lmP = groups["Close"].last()

# average price from start to (end − 30 mins)
avgP = groups.apply(lambda df: df["Close"].iloc[:-30].mean())

# binary label: 1 if avgP < lmP else 0
y = (avgP < lmP).astype(int)
y.name = "Y"

/tmp/ipykernel_101545/266240317.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avgP = groups.apply(lambda df: df["Close"].iloc[:-30].mean())


## 4. Feature Engineering
Here, new features are created for modeling purposes. Examples include previous price differences or multiple technical indicators.

In [33]:
# 3) —— Technical‐Indicator Functions —— #

# Basic EMAs & SMAs
def SMA(x, n):
    return x.rolling(n, min_periods=n).mean()

def EMA(x, n):
    return x.ewm(span=n, adjust=False).mean()

def DEMA(x, n):
    e = EMA(x, n)
    return 2*e - EMA(e, n)

# True Range & ATR
def ATR(H, L, C, n=14):
    prevC = C.shift(1)
    tr = pd.concat([
        H - L,
        (H - prevC).abs(),
        (L - prevC).abs()
    ], axis=1).max(axis=1)
    return EMA(tr, n)

# Average Directional Index (ADX)
def ADX(H, L, C, n=14):
    up = H.diff()
    down = -L.diff()
    plusDM  = np.where((up>down)&(up>0), up, 0.0)
    minusDM = np.where((down>up)&(down>0), down, 0.0)
    tr = pd.concat([
        H-L,
        (H - C.shift(1)).abs(),
        (L - C.shift(1)).abs()
    ], axis=1).max(axis=1)
    atr = EMA(tr, n)
    plusDI  = 100 * EMA(pd.Series(plusDM, index=H.index), n) / atr
    minusDI = 100 * EMA(pd.Series(minusDM, index=H.index), n) / atr
    dx = 100 * ( (plusDI - minusDI).abs() / (plusDI + minusDI) )
    return EMA(dx, n)

In [34]:
# Aroon
def aroon(xH, xL, n=14):
    def _aroon_up(series):
        idx = series.argmax()
        return ((n - (len(series)-1 - idx)) / n) * 100
    def _aroon_dn(series):
        idx = series.argmin()
        return ((n - (len(series)-1 - idx)) / n) * 100

    au = xH.rolling(n).apply(_aroon_up, raw=True)
    ad = xL.rolling(n).apply(_aroon_dn, raw=True)
    return pd.DataFrame({"aroonUp": au, "aroonDn": ad})

# Bollinger Bands
def BBands(x, n=20, k=2):
    m = SMA(x, n)
    sd = x.rolling(n).std()
    return pd.DataFrame({
        "bb_up":   m + k*sd,
        "bb_mid":  m,
        "bb_low":  m - k*sd
    })

# Commodity Channel Index
def CCI(H, L, C, n=20):
    TP = (H + L + C) / 3
    M  = SMA(TP, n)
    MD = TP.rolling(n).apply(lambda s: np.mean(np.abs(s - s.mean())), raw=True)
    return (TP - M) / (0.015 * MD)

# Chaikin Volatility
def chaikin_volatility(H, L, n=10, ema_n=10):
    hl = H - L
    e1 = EMA(hl, ema_n)
    e2 = e1.shift(n)
    return (e1 - e2) / e2 * 100

# Close Location Value
def CLV(H, L, C):
    return ((C - L) - (H - C)) / (H - L)

In [35]:
# Chande Momentum Oscillator
def CMO(x, n=14):
    diff = x.diff()
    up = diff.where(diff>0, 0.0).rolling(n).sum()
    dn = (-diff).where(diff<0, 0.0).rolling(n).sum()
    return 100 * (up - dn) / (up + dn)

# Centre of Gravity Indicator (CTI)
#  TTR’s CTI uses n=10 by default: CTI = (C - SMA(C,n)) / SMA(abs(C - SMA(C,n)),n)
def CTI(C, n=10):
    m = SMA(C, n)
    dev = (C - m).abs()
    return (C - m) / SMA(dev, n)

# Donchian Channel
def DonchianChannel(H, L, n=20):
    up = H.rolling(n).max()
    dn = L.rolling(n).min()
    mid = (up + dn) / 2
    return pd.DataFrame({"dc_up": up, "dc_mid": mid, "dc_dn": dn})

# Detrended Price Oscillator
def DPO(x, n=20):
    m = SMA(x, n)
    shift = int(n/2 + 1)
    return x.shift(shift) - m

# Directional Volatility Index (DVI)
#  TTR’s DVI default n=14: DVI = ROC of range
def DVI(C, n=14):
    rng = C.diff().abs()
    prev = rng.shift(n)

    # safe division: if prev==0 → NaN, else compute as usual
    dvi = (rng - prev) / prev * 100
    dvi = dvi.where(prev != 0)   # sets those inf entries to NaN

    return dvi

In [36]:
# Guppy Multiple Moving Average
def GMMA(x):
    short = [3,5,8,10,12,15]
    long  = [30,35,40,45,50,60]
    df = {}
    for i in short: df[f"gmma_s{i}"] = EMA(x, i)
    for i in long:  df[f"gmma_l{i}"] = EMA(x, i)
    return pd.DataFrame(df)

# Know Sure Thing
def KST(C, r1=10, r2=15, r3=20, r4=30, 
        n1=10, n2=10, n3=10, n4=15):
    roc1 = C.diff(r1)/C.shift(r1)
    roc2 = C.diff(r2)/C.shift(r2)
    roc3 = C.diff(r3)/C.shift(r3)
    roc4 = C.diff(r4)/C.shift(r4)
    kst = (SMA(roc1, n1)*1 +
           SMA(roc2, n2)*2 +
           SMA(roc3, n3)*3 +
           SMA(roc4, n4)*4)
    signal = SMA(kst, 9)
    return pd.DataFrame({"kst": kst, "signal": signal})

# Lagged differences
def lags(H, L, C, n=1):
    # show lagged close differences by default
    return C.diff(n)

# MACD
def MACD(C, n_fast=12, n_slow=26, n_sig=9):
    fast = EMA(C, n_fast)
    slow = EMA(C, n_slow)
    macd = fast - slow
    sig  = EMA(macd, n_sig)
    hist = macd - sig
    return pd.DataFrame({"macd": macd, "signal": sig, "hist": hist})

In [37]:
# Price Bands (percent)
def PBands(C, n=20, pct=0.025):
    m = SMA(C, n)
    return pd.DataFrame({
        "pb_up": m*(1+pct),
        "pb_mid": m,
        "pb_dn": m*(1-pct)
    })

# Rate of Change
def ROC(C, n=1):
    return (C - C.shift(n)) / C.shift(n) * 100

# Momentum
def momentum(C, n=1):
    return C - C.shift(n)

# Relative Strength Index
def RSI(C, n=14):
    diff = C.diff()
    up = diff.where(diff>0, 0.0)
    dn = -diff.where(diff<0, 0.0)
    ma_up = EMA(up, n)
    ma_dn = EMA(dn, n)
    rs = ma_up / ma_dn
    return 100 - (100 / (1 + rs))

# Running stats
def runSum(C, n=10):    return C.rolling(n).sum()
def runMin(C, n=10):    return C.rolling(n).min()
def runMax(C, n=10):    return C.rolling(n).max()
def runMedian(C,n=10):  return C.rolling(n).median()

In [38]:
# Parabolic SAR
def SAR(H, L, af0=0.02, af_step=0.02, af_max=0.2):
    up = True
    af = af0
    ep = H[0]
    sar = [L.iloc[0]]
    for i in range(1, len(H)):
        prev_sar = sar[-1]
        if up:
            new_sar = prev_sar + af*(ep - prev_sar)
            new_sar = min(new_sar, L.iloc[i-1], L.iloc[i])
            if H.iloc[i] > ep:
                ep = H.iloc[i]; af = min(af+af_step, af_max)
            if L.iloc[i] < new_sar:
                up = False; sar.append(ep); ep = L.iloc[i]; af = af0
            else:
                sar.append(new_sar)
        else:
            new_sar = prev_sar + af*(ep - prev_sar)
            new_sar = max(new_sar, H.iloc[i-1], H.iloc[i])
            if L.iloc[i] < ep:
                ep = L.iloc[i]; af = min(af+af_step, af_max)
            if H.iloc[i] > new_sar:
                up = True; sar.append(ep); ep = H.iloc[i]; af = af0
            else:
                sar.append(new_sar)
    return pd.Series(sar, index=H.index)

# Volatility (rolling stdev of log‐returns)
def volatility(H, L, C, n=10):
    lr = np.log(C / C.shift(1))
    return lr.rolling(n).std() * np.sqrt(252* (390/ (16*60)))  # annualize on minute data

# Ultimate Oscillator
def ultimate_oscillator(H, L, C, s1=7, s2=14, s3=28, w1=4, w2=2, w3=1):
    bp = C - np.minimum(L, C.shift(1))
    tr = np.maximum(H - L, np.maximum((H-C.shift(1)).abs(), (L-C.shift(1)).abs()))
    avg1 = bp.rolling(s1).sum() / tr.rolling(s1).sum()
    avg2 = bp.rolling(s2).sum() / tr.rolling(s2).sum()
    avg3 = bp.rolling(s3).sum() / tr.rolling(s3).sum()
    uo = 100 * (w1*avg1 + w2*avg2 + w3*avg3) / (w1+w2+w3)
    return uo

In [39]:
# Vertical–Horizontal Filter
def VHF(C, n=28):
    num = C.rolling(n).max() - C.rolling(n).min()
    den = C.diff().abs().rolling(n).sum()
    return num / den

# Williams Accumulation/Distribution
def williamsAD(H, L, C):
    clv = ((C - L) - (H - C)) / (H - L)
    return clv  # as in TTR

# Williams %R
def WPR(H, L, C, n=14):
    highest = H.rolling(n).max()
    lowest = L.rolling(n).min()
    return (highest - C) / (highest - lowest) * -100

# ZigZag
def ZigZag(H, L, C, pct=0.05):
    zz = [np.nan] * len(C)
    last_pivot = C.iloc[0]
    last_dir = None

    for i in range(1, len(C)):
        change = (C.iloc[i] - last_pivot) / last_pivot
        if last_dir is None:
            if abs(change) > pct:
                last_dir = np.sign(change)
                last_pivot = C.iloc[i]
                zz[i] = last_pivot
        else:
            if last_dir > 0 and C.iloc[i] > last_pivot:
                last_pivot = C.iloc[i]
                zz[i] = last_pivot
            elif last_dir < 0 and C.iloc[i] < last_pivot:
                last_pivot = C.iloc[i]
                zz[i] = last_pivot
            # reversal?
            elif (last_dir > 0 and (C.iloc[i] - last_pivot) / last_pivot < -pct) or \
                 (last_dir < 0 and (C.iloc[i] - last_pivot) / last_pivot > pct):
                last_dir *= -1
                last_pivot = C.iloc[i]
                zz[i] = last_pivot

    return pd.Series(zz, index=C.index)

In [40]:
# --- Triple Exponential Average oscillator (TRIX) ---
def TRIX(C, n=9):
    ema1 = EMA(C, n)
    ema2 = EMA(ema1, n)
    ema3 = EMA(ema2, n)
    trix = ema3.pct_change() * 100
    return trix

# --- Traders Dynamic Index (TDI) ---
def TDI(C, n_rsi=13, n_sig=2, n_bb=34, sd=1.618):
    rsi = RSI(C, n_rsi)
    signal = SMA(rsi, n_sig)
    bb    = BBands(rsi, n_bb, sd)
    return pd.DataFrame({
        "rsi":    rsi,
        "signal": signal,
        "bb_up":  bb["bb_up"],
        "bb_mid": bb["bb_mid"],
        "bb_dn":  bb["bb_low"]
    })

# --- Stochastic Momentum Index (SMI) ---
def SMI(H, L, C, n=14, n_fast=3, n_slow=25, n_sig=9):
    
    # median price and half‐range
    m = (H.rolling(n).max() + L.rolling(n).min()) / 2
    d = (H.rolling(n).max() - L.rolling(n).min()) / 2

    # double‐smoothed numerator and denominator
    num = EMA(EMA(C - m, n_fast), n_slow)
    den = EMA(EMA(d,       n_fast), n_slow)

    smi    = 100 * (num / den)
    signal = EMA(smi, n_sig)

    return pd.DataFrame({"smi": smi, "signal": signal})

In [41]:
# 4) —— Compute & aggregate “first-n-bar” daily averages —— #

def compute_daily_averages(spx):
    H, L, C, O = spx["High"], spx["Low"], spx["Close"], spx["Open"]
    n_days = len(y)
    # indices per day
    day_slices = []
    for date, df in spx.groupby("Date"):
        arr = df.index
        day_slices.append((arr[0], arr[-31]))  # from first bar up to bar-30
    
    # precompute all indicator series
    inds = {
        "avg_ADX":               ADX(H,L,C),
        "avg_aroonUp":           aroon(H,L)["aroonUp"],
        "avg_aroonDn":           aroon(H,L)["aroonDn"],
        "avg_ATR":               ATR(H,L,C),
        "avg_BBands_up":         BBands(C)["bb_up"],
        "avg_BBands_mid":        BBands(C)["bb_mid"],
        "avg_BBands_dn":         BBands(C)["bb_low"],
        "avg_CCI":               CCI(H,L,C),
        "avg_chaikinVolatility": chaikin_volatility(H,L),
        "avg_CLV":               CLV(H,L,C),
        "avg_CMOClose":          CMO(C),
        "avg_CTI":               CTI(C),
        "avg_Donchian_up":       DonchianChannel(H,L)["dc_up"],
        "avg_Donchian_mid":      DonchianChannel(H,L)["dc_mid"],
        "avg_Donchian_dn":       DonchianChannel(H,L)["dc_dn"],
        "avg_DPOClose":          DPO(C),
        "avg_DVIClose":          DVI(C),
        "avg_GMMAClose":         GMMA(C).mean(axis=1),
        "avg_KSTClose":          KST(C)["kst"],
        "avg_lagsClose":         lags(H,L,C),
        "avg_MACD":              MACD(C)["macd"],
        "avg_PBands_up":         PBands(C)["pb_up"],
        "avg_PBands_mid":        PBands(C)["pb_mid"],
        "avg_PBands_dn":         PBands(C)["pb_dn"],
        "avg_ROCClose":          ROC(C),
        "avg_momentumClose":     momentum(C),
        "avg_RSIClose":          RSI(C),
        "avg_runSum":            runSum(C, 10),
        "avg_runMin":            runMin(C, 10),
        "avg_runMax":            runMax(C, 10),
        "avg_runMedian":         runMedian(C, 10),
        "avg_SAR":               SAR(H,L),
        "avg_SMAClose":          SMA(C, 20),
        "avg_EMAClose":          EMA(C, 20),
        "avg_DEMAClose":         DEMA(C, 20),
        "avg_SNR":               VHF(C,28),   # approximate SNR via VHF
        "avg_SMI":               SMI(H,L,C),
        "avg_TDI":               TDI(C),    
        "avg_TRIX":              TRIX(C),   
        "avg_ultimateOscillator":ultimate_oscillator(H,L,C),
        "avg_VHF":               VHF(C),
        "avg_volatility":        volatility(H,L,C),
        "avg_williamsAD":        williamsAD(H,L,C),
        "avg_WPR":               WPR(H,L,C),
        "avg_ZigZag":            ZigZag(H,L,C)
    }

    data = {}
    for name, series in inds.items():
        # skip unimplemented features
        if series is None: 
            continue

        if isinstance(series, pd.DataFrame):
            # for each column in the DataFrame, make a separate feature
            for col in series.columns:
                key = f"{name}_{col}"
                data[key] = [
                    series[col].loc[start:end].mean()
                    for start, end in day_slices
                ]
        else:
            data[name] = [
                series.loc[start:end].mean()
                for start, end in day_slices
            ]

    return pd.DataFrame(data, index=y.index)

features = compute_daily_averages(spx)
features["Y"] = y
features.to_csv(OUTPUT_FEATURE_CSV, index_label="Date")

/tmp/ipykernel_101545/3631590232.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ep = H[0]


In [42]:
# 5) —— Train/Test split & Modeling —— #

# drop any rows with NA
allset = features.dropna().reset_index(drop=True)
allset.to_csv(OUTPUT_ALLSET_CSV, index=False)

X = allset.drop("Y", axis=1).values
y_ = allset["Y"].values
nx = len(allset)
train_n = int(np.floor(nx * 2/3))

X_train, X_test = X[:train_n], X[train_n:]
y_train, y_test = y_[:train_n], y_[train_n:]

In [43]:
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s  = scaler.transform(X_test)

def eval_model(X_tr, y_tr, X_te, y_te):
    rf = RandomForestClassifier(n_estimators=100, random_state=1, n_jobs=-1)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    scores = cross_val_score(rf, X_tr, y_tr, cv=cv, scoring="accuracy")
    rf.fit(X_tr, y_tr)
    y_pred = rf.predict(X_te)
    cm = confusion_matrix(y_te, y_pred)
    return {
        "accuracy": accuracy_score(y_te, y_pred),
        "precision": precision_score(y_te, y_pred, pos_label=0),
        "recall": recall_score(y_te, y_pred, pos_label=0),
        "f1": f1_score(y_te, y_pred, pos_label=0),
        "cm": cm
    }

## 5. Resampling Techniques & Model Training
We apply methods such as SMOTE, random oversampling, and undersampling to address class imbalance in the dataset.
SMOTE (Synthetic Minority Over-sampling Technique) creates synthetic samples of the minority class by interpolating between exsisting, random oversampling duplicates existing samples from the minority class, while undersampling reduces the majority class to balance the dataset.

A Random Forest Classifier is trained using k-fold cross-validation. The performance is evaluated on key metrics: accuracy, precision, recall, and F1 score.

In [ ]:
# original
results = {}
results["original"] = eval_model(X_train_s, y_train, X_test_s, y_test)

# undersample
rus = RandomUnderSampler(random_state=1)
X_ru, y_ru = rus.fit_resample(X_train_s, y_train)
results["under"] = eval_model(X_ru, y_ru, X_test_s, y_test)

# oversample
ros = RandomOverSampler(random_state=1)
X_ro, y_ro = ros.fit_resample(X_train_s, y_train)
results["over"] = eval_model(X_ro, y_ro, X_test_s, y_test)

# SMOTE
sm = SMOTE(random_state=1)
X_sm, y_sm = sm.fit_resample(X_train_s, y_train)
results["smote"] = eval_model(X_sm, y_sm, X_test_s, y_test)

# print comparison
comp = pd.DataFrame({
    m: {
        "Accuracy": results[m]["accuracy"],
        "Precision": results[m]["precision"],
        "Recall": results[m]["recall"],
        "F1": results[m]["f1"]
    } for m in results
}).T
print(comp)

comp.to_csv("../csvfiles_python/comparison_360_raw.csv")

          Accuracy  Precision    Recall        F1
original  0.737789   0.709302  0.701149  0.705202
under     0.709512   0.654822  0.741379  0.695418
over      0.706941   0.678571  0.655172  0.666667
smote     0.727506   0.712500  0.655172  0.682635


## 8. Summary of Results (Short Version)
Among all sampling methods tested, the original dataset produced the best overall performance in terms of accuracy and F1 score, suggesting that resampling was not strictly necessary. SMOTE offered a balanced improvement in recall without sacrificing much precision, making it a good alternative if identifying positive signals is a priority. Undersampling achieved the highest recall but at the cost of precision, while oversampling showed limited benefit in this case.

## 9. Final Notes
Further improvement could involve using more advanced models (e.g., XGBoost) or time-series-specific approaches (e.g., LSTM).